In [ ]:
%load_ext kedro.ipython
%reload_kedro

In [ ]:
from scipy.cluster.hierarchy import cut_tree
from tqdm.auto import tqdm
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import matplotlib
from matplotlib.figure import Figure
from srai_kedro.pipelines.processing.nodes import _load_gdf
from typing import Tuple

In [ ]:
regions_clustered = catalog.load("regions_clustered")
regions_clustered

In [ ]:
clusters = catalog.load("params:clustering.clusters")
clusters

In [ ]:
roads = catalog.load("roads_concat")
roads

In [ ]:
areas = catalog.load("areas")
areas = {k: _load_gdf(v) for k, v in areas.items()}
areas

In [ ]:
MAP_SOURCE = ctx.providers.CartoDB.Positron
MATPLOTLIB_COLORMAP = 'tab20'
PLOTLY_COLORMAP = list(map(lambda color: f"rgb{tuple(map(lambda color_compound: color_compound * 255, color))}", matplotlib.colormaps[MATPLOTLIB_COLORMAP].colors))


def pyplot_clustered_regions_with_roads(regions: gpd.GeoDataFrame, roads: gpd.GeoDataFrame, column: str, title: str = "") -> Tuple[Figure, plt.Axes]:
    fig, ax = plt.subplots(figsize=(10, 9))
    ax.set_aspect('equal')
    ax.set_title(title)
    regions.to_crs(epsg=3857).plot(column=column, ax=ax, alpha=0.9, legend=True, cmap=MATPLOTLIB_COLORMAP, vmin=0, vmax=len(PLOTLY_COLORMAP), linewidth=0)
    roads.to_crs(epsg=3857).plot(ax=ax, color="black", alpha=0.5, linewidth=0.2)
    ctx.add_basemap(ax, source=MAP_SOURCE)
    return fig, ax


area = areas["Wroclaw_Poland"]

for idx, c in enumerate(clusters):
    fig, ax = pyplot_clustered_regions_with_roads(regions_clustered.sjoin(area), roads.sjoin(area), f"cluster_{c}", title=f"cluster_{c}")
    ax.set_axis_off()
    plt.tight_layout()